In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [3]:
TRAIN_PATH = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt"
train_data = pd.read_csv(TRAIN_PATH, 
                delimiter=":::", names=["Id", "Title", "Genre", "Description"])

<ipython-input-3-54e0d71308ee>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_data = pd.read_csv(TRAIN_PATH,


In [4]:
train_data = train_data.drop("Id", axis=1)

In [5]:
train_data

,Title,Genre,Description
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...
54209,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [6]:
train_data.isna().sum()

Title          0
Genre          0
Description    0
dtype: int64

In [7]:
# For memory purpose we will use only the first 1000
train_data = train_data.iloc[:1000]

In [8]:
tfid = TfidfVectorizer()

In [9]:
import string
import nltk

def clean_description(description):
    description = re.sub(r'@\S+', '', description.lower())
    description = re.sub(r'http\S+', '', description) 
    description = re.sub(r'pic.\S+', '', description)
    description = re.sub(r"[^a-zA-Z+']", ' ', description)  
    description = re.sub(r'\s+[a-zA-Z]\s+', ' ', description + ' ')
    stopwords = nltk.corpus.stopwords.words('english')  
    description = "".join([w for w in description if w not in string.punctuation ])
    words = nltk.word_tokenize(description)
    description = " ".join([w for w in words if w not in stopwords ])

    return description

In [10]:
clean_description(train_data["Description"][0])

'listening conversation doctor parents year old oscar learns nobody courage tell weeks live furious refuses speak anyone except straight talking rose lady pink meets hospital stairs christmas approaches rose uses fantastical experiences professional wrestler imagination wit charm allow oscar live life love full company friends pop corn einstein bacon childhood sweetheart peggy blue'

In [11]:
train_data["Description"] = train_data["Description"].apply(clean_description)

x = tfid.fit_transform(train_data["Description"]).toarray()
y = train_data["Genre"].values

x.shape, y.shape

<ipython-input-11-e6a2b058d1a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Description"] = train_data["Description"].apply(clean_description)


((1000, 14523), (1000,))

In [12]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.2, random_state=12)

In [13]:
from sklearn.svm import SVC

svc = SVC()

In [14]:
svc.fit(x_train, y_train)

SVC()

In [15]:
print("model score: ",svc.score(x_train, y_train))

model score:  0.80125


In [16]:
from sklearn.metrics import accuracy_score 
prediction = svc.predict(x_test)
print("Accuracy score: ", accuracy_score(prediction, y_test))

Accuracy score:  0.48
